In [11]:
import sys,os
__script_path=os.path.abspath(globals().get('__file__','.'))
__script_dir = os.path.dirname(__script_path)
root_dir = os.path.abspath(f'{__script_dir}/../../..')
print(root_dir)
for lib in [root_dir][::-1]:
    if lib in sys.path:
        sys.path.remove(lib)
    sys.path.insert(0,lib)

from libs.common import *
from config.config import *

c:\Users\Admin\Data\ads_fraud_detection


# 3. Tiền xử lí dữ liệu

In [ ]:
save_dir=f"{exps_dir}/exp3/exp_gau"

save_dir = f"{save_dir}"
os.makedirs(save_dir, exist_ok=True)
test_size=0.33
seed=42

In [13]:
test_size=0.33
seed=42
train=pd.read_csv(f'{exps_dir}/exp3/train_EDA.csv')
test=pd.read_csv(f'{exps_dir}/exp3/test_EDA.csv')
train.head()

,isFraud,transactionamt,productcd,card1,card2,card3,card5,card6,addr1,dist1,p_emaildomain,r_emaildomain,c1,c2,c4,c5,c6,c7,c8,c9,c10,c11,c12,c13,c14,v62,v70,v76,v78,v82,v91,v127,v130,v139,v160,v165,v187,v203,v207,v209,v210,v221,v234,v257,v258,v261,v264,v266,v267,v271,v274,v277,v283,v285,v289,v291,v294,id_01,id_02,id_05,id_06,id_09,id_13,id_17,id_19,id_20,devicetype,deviceinfo,EVENT_ID,ENTITY_ID,EVENT_TIMESTAMP,ENTITY_TYPE,transactionamt_to_mean_card1,transactionamt_to_mean_card2,transactionamt_to_std_card1,transactionamt_to_mean_card3,transactionamt_to_std_card3,transactionamt_to_mean_card5,transactionamt_to_std_card5,transactionamt_to_mean_card6,transactionamt_to_std_card6,transactionamt_to_std_card2,id_02_to_mean_card1,id_02_to_std_card1,id_02_to_mean_card2,id_02_to_std_card2,id_02_to_mean_card3,id_02_to_std_card3,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,P_emaildomain_3,R_emaildomain_1,R_emaildomain_2,R_emaildomain_3
0,0,68.5,W,13926.0,NaN,150.0,142.0,credit,315.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,117.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d513a58e-ef4b-4025-b511-5e3f54eb6934,13926.0_315.0_-13.0,2021-01-02T00:00:00Z,user,0.194640,NaN,0.184566,0.468544,0.273939,0.350242,0.202795,0.363321,0.203233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,150.0,102.0,credit,325.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4ecf67fc-3355-45a8-8786-412b2ed8e301,2755.0_325.0_1.0,2021-01-02T00:00:01Z,user,0.122385,0.126450,0.061803,0.198362,0.115974,0.135734,0.066448,0.153815,0.086040,0.079439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN,NaN,NaN
2,0,59.0,W,4663.0,490.0,150.0,166.0,debit,330.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6d0c2fd4-6e2d-4d7b-9518-c36bc29ffb3a,4663.0_330.0_1.0,2021-01-02T00:01:09Z,user,0.606546,0.444228,0.580118,0.403564,0.235948,0.601891,0.435485,0.507196,0.310544,0.283613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN,NaN,NaN
3,0,50.0,W,18132.0,567.0,150.0,117.0,debit,476.0,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,4.0,0.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1758.0,354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,0.0,1.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,590d0c68-bdb1-4d9d-b497-498709679a7f,18132.0_476.0_-111.0,2021-01-02T00:01:39Z,user,0.410485,0.381136,0.272005,0.342003,0.199956,0.398820,0.257697,0.429827,0.263173,0.254018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN,NaN,NaN
4,0,50.0,H,4497.0,514.0,150.0,102.0,credit,420.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,169690.8,5155.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,70787.0,NaN,NaN,NaN,NaN,166.0,542.0,144.0,mobile,SAMSUNG SM-G892A Build/NRD90M,c60dd336-871a-4515-a6ad-56e0bdeee79c,4497.0_420.0_1.0,2021-01-02T00:01:46Z,user,0.515612,0.222350,0.882933,0.342003,0.199956,0.234025,0.114565,0.265198,0.148345,0.093310,0.764773,1.753301,0.579193,0.626598,0.604509,0.585955,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN,NaN,NaN


In [14]:
one_value_cols = [col for col in train.columns if train[col].nunique() <= 1]
one_value_cols_test = [col for col in test.columns if test[col].nunique() <= 1]
one_value_cols == one_value_cols_test

True

In [15]:
many_null_cols = [col for col in train.columns if train[col].isnull().sum() / train.shape[0] > 0.9]
many_null_cols_test = [col for col in test.columns if test[col].isnull().sum() / test.shape[0] > 0.9]

In [16]:
big_top_value_cols = [col for col in train.columns if train[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]
big_top_value_cols_test = [col for col in test.columns if test[col].value_counts(dropna=False, normalize=True).values[0] > 0.9]

In [17]:
cols_to_drop = list(set(many_null_cols + many_null_cols_test + big_top_value_cols + big_top_value_cols_test + one_value_cols+ one_value_cols_test))
print(cols_to_drop)
cols_to_drop.remove('isFraud')
len(cols_to_drop)

['isFraud', 'ENTITY_TYPE', 'c7', 'P_emaildomain_3', 'R_emaildomain_3', 'card3']


5

In [22]:
train.shape, test.shape

((561013, 93), (29527, 94))

In [23]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)

In [26]:
train.head()

,isFraud,transactionamt,productcd,card1,card2,card5,card6,addr1,dist1,p_emaildomain,r_emaildomain,c1,c2,c4,c5,c6,c8,c9,c10,c11,c12,c13,c14,v62,v70,v76,v78,v82,v91,v127,v130,v139,v160,v165,v187,v203,v207,v209,v210,v221,v234,v257,v258,v261,v264,v266,v267,v271,v274,v277,v283,v285,v289,v291,v294,id_01,id_02,id_05,id_06,id_09,id_13,id_17,id_19,id_20,devicetype,deviceinfo,EVENT_ID,ENTITY_ID,EVENT_TIMESTAMP,transactionamt_to_mean_card1,transactionamt_to_mean_card2,transactionamt_to_std_card1,transactionamt_to_mean_card3,transactionamt_to_std_card3,transactionamt_to_mean_card5,transactionamt_to_std_card5,transactionamt_to_mean_card6,transactionamt_to_std_card6,transactionamt_to_std_card2,id_02_to_mean_card1,id_02_to_std_card1,id_02_to_mean_card2,id_02_to_std_card2,id_02_to_mean_card3,id_02_to_std_card3,id_02_to_mean_card5,id_02_to_std_card5,id_02_to_mean_card6,id_02_to_std_card6,P_emaildomain_1,P_emaildomain_2,R_emaildomain_1,R_emaildomain_2
0,0,68.5,W,13926.0,NaN,142.0,credit,315.0,19.0,NaN,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,117.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,d513a58e-ef4b-4025-b511-5e3f54eb6934,13926.0_315.0_-13.0,2021-01-02T00:00:00Z,0.194640,NaN,0.184566,0.468544,0.273939,0.350242,0.202795,0.363321,0.203233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,29.0,W,2755.0,404.0,102.0,credit,325.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4ecf67fc-3355-45a8-8786-412b2ed8e301,2755.0_325.0_1.0,2021-01-02T00:00:01Z,0.122385,0.126450,0.061803,0.198362,0.115974,0.135734,0.066448,0.153815,0.086040,0.079439,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gmail,com,NaN,NaN
2,0,59.0,W,4663.0,490.0,166.0,debit,330.0,287.0,outlook.com,NaN,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6d0c2fd4-6e2d-4d7b-9518-c36bc29ffb3a,4663.0_330.0_1.0,2021-01-02T00:01:09Z,0.606546,0.444228,0.580118,0.403564,0.235948,0.601891,0.435485,0.507196,0.310544,0.283613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,outlook,com,NaN,NaN
3,0,50.0,W,18132.0,567.0,117.0,debit,476.0,NaN,yahoo.com,NaN,2.0,5.0,0.0,0.0,4.0,0.0,1.0,0.0,1.0,0.0,25.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1758.0,354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,10.0,0.0,1.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,590d0c68-bdb1-4d9d-b497-498709679a7f,18132.0_476.0_-111.0,2021-01-02T00:01:39Z,0.410485,0.381136,0.272005,0.342003,0.199956,0.398820,0.257697,0.429827,0.263173,0.254018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yahoo,com,NaN,NaN
4,0,50.0,H,4497.0,514.0,102.0,credit,420.0,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,169690.8,5155.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,70787.0,NaN,NaN,NaN,NaN,166.0,542.0,144.0,mobile,SAMSUNG SM-G892A Build/NRD90M,c60dd336-871a-4515-a6ad-56e0bdeee79c,4497.0_420.0_1.0,2021-01-02T00:01:46Z,0.515612,0.222350,0.882933,0.342003,0.199956,0.234025,0.114565,0.265198,0.148345,0.093310,0.764773,1.753301,0.579193,0.626598,0.604509,0.585955,0.50153,0.487971,0.442759,0.470755,gmail,com,NaN,NaN


In [27]:
train.isnull().sum()

isFraud                              0
transactionamt                       0
productcd                            0
card1                                0
card2                             8604
card5                             4158
card6                             1570
addr1                            62884
dist1                           336363
p_emaildomain                    89264
r_emaildomain                   429350
c1                                   0
c2                                   0
c4                                   0
c5                                   0
c6                                   0
c8                                   0
c9                                   0
c10                                  0
c11                                  0
c12                                  0
c13                                  0
c14                                  0
v62                              74688
v70                              74688
v76                      

In [29]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns
categorical_cols

Index(['productcd', 'card6', 'p_emaildomain', 'r_emaildomain', 'devicetype', 'deviceinfo', 'EVENT_ID', 'ENTITY_ID', 'EVENT_TIMESTAMP', 'P_emaildomain_1', 'P_emaildomain_2', 'R_emaildomain_1', 'R_emaildomain_2'], dtype='object')

In [ ]:
continuous_cols = train.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = train.select_dtypes(include=['object']).columns

for col in continuous_cols:
    train[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    train[col].fillna(train[col].mode()[0], inplace=True)

train.isnull().sum()

isFraud                          0
transactionamt                   0
productcd                        0
card1                            0
card2                            1
card5                            0
card6                            0
addr1                            0
dist1                            0
p_emaildomain                    0
r_emaildomain                    0
c1                               0
c2                               0
c4                               0
c5                               0
c6                               0
c8                               0
c9                               0
c10                              0
c11                              0
c12                              0
c13                              0
c14                              0
v62                              0
v70                              0
v76                              0
v78                              0
v82                              0
v91                 

In [19]:
train = train.drop(cols_to_drop, axis=1)
test = test.drop(cols_to_drop, axis=1)

In [31]:
cat_cols = ['id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28', 'id_29',
            'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo', 'ProductCD', 'card4', 'card6', 'M4','P_emaildomain',
            'R_emaildomain', 'card1', 'card2', 'card3',  'card5', 'addr1', 'addr2', 'M1', 'M2', 'M3', 'M5', 'M6', 'M7', 'M8', 'M9',
            'P_emaildomain_1', 'P_emaildomain_2', 'P_emaildomain_3', 'R_emaildomain_1', 'R_emaildomain_2', 'R_emaildomain_3']
for col in cat_cols:
    if col in train.columns:
        le = LabelEncoder()
        le.fit(list(train[col].astype(str).values) + list(test[col].astype(str).values))
        train[col] = le.transform(list(train[col].astype(str).values))
        test[col] = le.transform(list(test[col].astype(str).values))   

In [34]:
train.columns

Index(['isFraud', 'transactionamt', 'productcd', 'card1', 'card2', 'card5', 'card6', 'addr1', 'dist1', 'p_emaildomain', 'r_emaildomain', 'c1', 'c2', 'c4', 'c5', 'c6', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'v62', 'v70', 'v76', 'v78', 'v82', 'v91', 'v127', 'v130', 'v139', 'v160', 'v165', 'v187', 'v203', 'v207', 'v209', 'v210', 'v221', 'v234', 'v257', 'v258', 'v261', 'v264', 'v266', 'v267', 'v271', 'v274', 'v277', 'v283', 'v285', 'v289', 'v291', 'v294', 'id_01', 'id_02', 'id_05', 'id_06', 'id_09', 'id_13', 'id_17', 'id_19', 'id_20', 'devicetype', 'deviceinfo', 'EVENT_ID', 'ENTITY_ID', 'EVENT_TIMESTAMP', 'transactionamt_to_mean_card1', 'transactionamt_to_mean_card2', 'transactionamt_to_std_card1', 'transactionamt_to_mean_card3', 'transactionamt_to_std_card3', 'transactionamt_to_mean_card5', 'transactionamt_to_std_card5', 'transactionamt_to_mean_card6', 'transactionamt_to_std_card6', 'transactionamt_to_std_card2', 'id_02_to_mean_card1', 'id_02_to_std_card1', 'id_02_to_mean_card2',


In [ ]:
X_train = train.drop(['isFraud'], axis=1)
y_train = train['isFraud']


# del train
# test = test[["TransactionDT", 'TransactionID']]

In [37]:
y_train

0         0
1         0
2         0
3         0
4         0
         ..
561008    0
561009    0
561010    0
561011    0
561012    0
Name: isFraud, Length: 561013, dtype: int64

In [38]:
y_train.values

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [39]:
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
np.savez(f'{save_dir}/class_weights_dict.npz', class_weights_dict = class_weights_dict)
class_weights_dict

{0: 0.5179993684432125, 1: 14.389376218323587}

In [ ]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_rosampled, y_rosampled = ros.fit_resample(train.drop(columns=['isFraud']), train['isFraud'])
X_rosampled = pd.DataFrame(X_rosampled)
y_rosampled = pd.DataFrame(y_rosampled)
oversampling_sample=pd.concat([X_rosampled, y_rosampled], axis=1)    

In [ ]:
from sdv.metadata import SingleTableMetadata
metadata_sync = SingleTableMetadata()
metadata_sync.detect_from_dataframe(data=oversampling_sample)

In [ ]:
from sdv.single_table import GaussianCopulaSynthesizer



gau_synthesizer = GaussianCopulaSynthesizer(metadata_sync,
                                            enforce_min_max_values=True,  
                                            enforce_rounding=True,
                                            default_distribution='gamma') 

gau_synthesizer.fit(oversampling_sample)
gau_synthesizer.save(
    filepath=f'{save_dir}/gau_synthesizer.pkl'
)

In [ ]:
gau_synthetic_data = gau_synthesizer.sample(num_rows=len(train))
gau_synthetic_data.head()

In [ ]:
from sdv.evaluation.single_table import run_diagnostic



diagnostic_gau = run_diagnostic(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sdv.evaluation.single_table import evaluate_quality



diagnostic_gau = evaluate_quality(
    real_data=oversampling_sample,
    synthetic_data=gau_synthetic_data,
    metadata=metadata_sync
)




In [ ]:
from sklearn.utils import shuffle
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)

## Gau data
gau_data=gau_synthetic_data[gau_synthetic_data.isFraud==1]
gau_sample=pd.concat([train, gau_data] , axis=0, ignore_index=True)
X_gausampled, y_gausampled = rus.fit_resample(gau_sample.drop(columns=['isFraud']), gau_sample['isFraud'])
X_gausampled = pd.DataFrame(X_gausampled)
y_gausampled = pd.DataFrame(y_gausampled)
gau_sample=pd.concat([X_gausampled, y_gausampled], axis=1)    
gau_sample = shuffle(gau_sample, random_state=42)
print(gau_sample.isFraud.value_counts())

In [ ]:
from sklearn.model_selection import train_test_split
# x_train, x_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.20, random_state = 101)
X_train.to_csv(f'{save_dir}/x_train.csv', index=False)
y_train.to_csv(f'{save_dir}/y_train.csv', index=False)
# x_valid.to_csv(f'{save_dir}/x_valid.csv', index=False)
# y_valid.to_csv(f'{save_dir}/y_valid.csv', index=False)

In [40]:

for col in continuous_cols:
    test[col].interpolate(method='linear', inplace=True)

for col in categorical_cols:
    test[col].fillna(test[col].mode()[0], inplace=True)

x_test = test.drop(['isFraud'], axis=1)
y_test = test['isFraud']

x_test.to_csv(f'{save_dir}/x_test.csv', index=False)
y_test.to_csv(f'{save_dir}/y_test.csv', index=False)